In [43]:
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
import tqdm

myseed = 12345
torch.manual_seed(myseed)
torch.random.manual_seed(myseed)
random.seed(0)
np.random.seed(myseed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed_all(myseed)
torch.autograd.set_detect_anomaly(True)  # 可在NaN 出现时报错，定位错误代码。正向传播时：开启自动求导的异常侦测
# 反向传播时：在求导时开启侦测
#with torch.autograd.detect_anomaly():
#    loss.backward()
torch.multiprocessing.set_sharing_strategy('file_system')

In [44]:
# 使用 GPU
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [45]:
# 数据预处理, 调整图片大小
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize([224, 224])])

# 下载Fashion-MNIST训练集数据，并构建训练集数据载入器trainloader,每次从训练集中载入64张图片，每次载入都打乱顺序
trainset = torchvision.datasets.FashionMNIST('dataset/', download=False, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 下载Fashion-MNIST测试集数据，并构建测试集数据载入器trainloader,每次从测试集中载入64张图片，每次载入都打乱顺序
testset = torchvision.datasets.FashionMNIST('dataset/', download=False, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [46]:
class Denseblock(torch.nn.Module):
    def __init__(self, num_layers, in_channals, out_channals, strides = 1):
        super(Denseblock, self).__init__()
        self.net = torch.nn.Sequential()
        for i in range(num_layers):
            if not i:
                self.net.append(torch.nn.Sequential(
                        torch.nn.Conv2d(in_channals, out_channals, kernel_size=1),
                        torch.nn.ReLU(),
                        torch.nn.Conv2d(out_channals, out_channals, kernel_size=3, padding=1),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(out_channals)
                    )
                )
            else:
                self.net.append(torch.nn.Sequential(
                        torch.nn.Conv2d(out_channals, out_channals, kernel_size=1),
                        torch.nn.ReLU(),
                        torch.nn.Conv2d(out_channals, out_channals, kernel_size=3, padding=1),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(out_channals)
                    )
                )

    def forward(self, x):
        Y = self.net(x)
        X = torch.cat((Y, x), dim=1)
        return X

In [47]:
def transition_layer(in_channals, out_channals):
    return torch.nn.Sequential(
        torch.nn.Conv2d(in_channals, out_channals, kernel_size=1),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2)
    )

In [48]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 112, kernel_size=7, stride=2, padding=3), 
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        )
        self.layer2 = Denseblock(6, 112, 56)
        self.layer3 = transition_layer(168, 28)
        self.layer4 = Denseblock(12, 28, 28)
        self.layer5 = transition_layer(56, 14)
        self.layer6 = Denseblock(24, 14, 14)
        self.layer7 = transition_layer(28, 7)
        self.layer8 = Denseblock(16, 7, 7)
        self.layer9 = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((1, 1)),
            torch.nn.Flatten(),
            torch.nn.Linear(14, 64),
            torch.nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        return x

In [49]:
model = Net()
model = model.to(device=try_gpu())

# 定义优化器和损失函数
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

model.train()
epochs= 1

Loss_data = {
    "train": [],
    "dev": []
}

for epoch in tqdm.tqdm(range(epochs)):
    Loss = 0
    for batch_x, batch_y in trainloader:
        batch_x, batch_y = batch_x.to(device=try_gpu()), batch_y.to(device=try_gpu())
        prediction = model(batch_x)
        loss = criterion(prediction, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        Loss_data["train"].append(float(loss))
        Loss_data["dev"].append(0)

  0%|          | 0/1 [25:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    plt.figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(0.0, 5.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()
    
plot_learning_curve(Loss_data, "Loss")

In [ ]:
model.eval()
predicted_labels = []
epoch = 0
accuracy_sum = 0
with torch.no_grad():
    for batch_x, batch_y in testloader:
        batch_x, batch_y = batch_x.to(device = try_gpu()), batch_y.to(device = try_gpu())
        prediction = model(batch_x)
        predicted_label = torch.argmax(prediction,1)
        accuracy = torch.eq(batch_y, predicted_label).float().mean()
        accuracy_sum += accuracy
        epoch = epoch + 1

print("准确率: %f" % (accuracy / epoch))